In [68]:
import requests
from bs4 import BeautifulSoup
import re

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept-Language": "fr-FR,fr;q=0.9",
    "Referer": "https://www.paruvendu.fr/immobilier/annonces-immo/aquitaine/"
}
session = requests.Session()

def parse_card(card):
    # --- Titre ---
    h3 = card.select_one("h3")
    parts = [p.strip() for p in h3.get_text("\\n").split("\\n") if p.strip()] if h3 else []
    type_bien = parts[0] if len(parts) > 0 else None  # "Appartement"
    surface_brut   = parts[1] if len(parts) > 1 else None  # "41 m2"
    print(surface_brut)
    if surface_brut:
        surface_clean = int(re.sub(r"\D", "", surface_brut))
    else:
        surface_clean = None

    ville     = parts[2] if len(parts) > 2 else None  # "Saint-André-de-Cubzac (33)"
    # --- Prix (nettoyé en entier) ---
    price_el = card.select_one(".encoded-lnk")
    prix_raw = price_el.get_text(" ", strip=True) if price_el else None
    
    if prix_raw:
        prix_digits = "".join(filter(str.isdigit, prix_raw))
        prix = int(prix_digits) if prix_digits else None
        is_fai = "*" in prix_raw
    else:
        prix = None
        is_fai = False

    # --- Badges (pièces, chambres, équipements) ---
    badges = [b.get_text(" ", strip=True) for b in card.select("li.text-xs, span.text-xs")]
    # --- DPE (lettre A-G) ---
    dpe_span = card.find("span", class_=lambda c: c and "rounded-md" in c and "No" in c)
    dpe = dpe_span.get_text(strip=True) if dpe_span else None
    # --- Label image (Nouveau / Exclusif) ---
    label_el = card.select_one(".etiquetteAnnonce")
    label = label_el.get_text(strip=True) if label_el else None
    # --- Vendeur ---
    particulier = card.select_one(".pseudoinfo")
    agence_link = card.select_one("a[title*='annonces']")
    agence_name = card.select_one("a.text-sm.text-black.line-clamp-1")
    agent_el    = card.select_one("p.text-sm.text-grey-600.line-clamp-1")
    vendeur = "Particulier" if particulier else (
              agence_name.get_text(strip=True) if agence_name else None)
    agent   = agent_el.get_text(strip=True) if agent_el and not particulier else None
    # --- Date ---
    date_el = card.select_one("p.text-xs")
    date = date_el.get_text(" ", strip=True) if date_el else None
    # --- URL & ID ---
    link = card.select_one('a[href*="/immobilier/vente/"], a[href*="/immobilier/prestige/"]')
    url  = "https://www.paruvendu.fr" + link["href"] if link else None
    id_  = card.get("data-id")
    return {
        "id": id_, "type": type_bien, "surface": surface_clean, "ville": ville,
        "prix": prix, "prix_fai": is_fai, "dpe": dpe, "label": label,
        "badges": badges, "vendeur": vendeur, "agent": agent,
        "date": date, "url": url
    }

results = []

for page in range(1, 2):
    url = f"https://www.paruvendu.fr/immobilier/vente/aquitaine/?p={page}&allp=1"
    r = session.get(url, headers=headers, timeout=10)
    soup = BeautifulSoup(r.content, "html.parser", from_encoding="windows-1252")
    cards = soup.select(".blocAnnonce")
    for card in cards:
        results.append(parse_card(card))

print(len(results))
print(results)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
30
[{'id': '1288037278', 'type': 'Appartement\r\n41 m', 'surface': 2, 'ville': 'Saint-André-de-Cubzac (33)', 'prix': 139000, 'prix_fai': True, 'dpe': 'D', 'label': None, 'badges': ['Studio - 1 pièce', '1\r\nchambre', 'Garage', 'Balcon'], 'vendeur': 'Particulier', 'agent': None, 'date': '18/02/2026', 'url': 'https://www.paruvendu.fr/immobilier/vente/appartement/1288037278A1KIVHAP000'}, {'id': '1287465349', 'type': 'Maison\r\n274 m', 'surface': 2, 'ville': 'Habas (40)', 'prix': 399000, 'prix_fai': False, 'dpe': 'C', 'label': None, 'badges': ['9 pièces', '5\r\nchambres', 'Terrain 4 512 m 2'], 'vendeur': 'Optimhome', 'agent': 'Hélène Monnier', 'date': '18/02/2026', 'url': 'https://www.paruvendu.fr/immobilier/vente/maison/1287465349A1KIVHMN000'}, {'id': '1288616098', 'type': 'Appartement\r\n63 m', 'surface': 2, 'ville': 'Bordeaux (33)', 'prix': 270000, 'prix_fai': True, 'dpe': 'D', 'label': 'Nouveau', 'badges': ['2/3 pièces', '1\r\

In [56]:
import psycopg2

conn = psycopg2.connect(
    dbname="paruvendu",
    user="postgres",
    password="12365478901",
    host="localhost",
    port="5432"
)

cursor = conn.cursor();


In [24]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS biens (
    id          SERIAL PRIMARY KEY,
    annonce_id  VARCHAR(30),          -- data-id de la page (ex: "1288037278")
    type_bien   VARCHAR(100),
    surface     INT,                  -- stocker en nombre, ex: 41
    ville       VARCHAR(150),
    prix        INT,                  -- stocker en nombre, ex: 139000
    prix_fai    BOOLEAN,              -- TRUE si prix FAI (avec *)
    dpe         CHAR(1),              -- lettre A-G
    label       VARCHAR(50),          -- "Nouveau", "Exclusif", etc.
    badges      TEXT[],               -- tableau PostgreSQL : ARRAY['2 pièces', 'Garage']
    vendeur     VARCHAR(150),
    agent       VARCHAR(150),
    date_annonce DATE,
    url         TEXT                  -- TEXT, pas de type "url" en PostgreSQL
);
'''

cursor.execute(create_table_query)
conn.commit()
print("Table créée avec succès : biens")

Table créée avec succès : biens


In [65]:
def insert_bien(card):
    insert_query = '''
    INSERT INTO biens 
    (annonce_id, type_bien, surface, ville, prix, prix_fai, dpe, label, badges, vendeur, agent, date_annonce, url)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    RETURNING id;
    '''
    try:
        cursor.execute(insert_query, (
            card["id"],
            card["type"],
            card["surface"],
            card["ville"],
            card["prix"],
            card["prix_fai"],
            card["dpe"],
            card["label"],
            card["badges"],
            card["vendeur"],
            card["agent"],
            card["date"],
            card["url"]
        ))
        new_id = cursor.fetchone()[0]
        conn.commit()
        print("Bien enregistré avec l'ID :", new_id)

    except Exception as e:
        conn.rollback()
        print("Erreur insertion :", e)


for bien in results:
    insert_bien(bien)

Bien enregistré avec l'ID : 31229
Bien enregistré avec l'ID : 31230
Bien enregistré avec l'ID : 31231
Bien enregistré avec l'ID : 31232
Bien enregistré avec l'ID : 31233
Bien enregistré avec l'ID : 31234
Bien enregistré avec l'ID : 31235
Bien enregistré avec l'ID : 31236
Bien enregistré avec l'ID : 31237
Bien enregistré avec l'ID : 31238
Bien enregistré avec l'ID : 31239
Bien enregistré avec l'ID : 31240
Bien enregistré avec l'ID : 31241
Bien enregistré avec l'ID : 31242
Bien enregistré avec l'ID : 31243
Bien enregistré avec l'ID : 31244
Bien enregistré avec l'ID : 31245
Bien enregistré avec l'ID : 31246
Bien enregistré avec l'ID : 31247
Bien enregistré avec l'ID : 31248
Bien enregistré avec l'ID : 31249
Bien enregistré avec l'ID : 31250
Bien enregistré avec l'ID : 31251
Bien enregistré avec l'ID : 31252
Bien enregistré avec l'ID : 31253
Bien enregistré avec l'ID : 31254
Bien enregistré avec l'ID : 31255
Bien enregistré avec l'ID : 31256
Bien enregistré avec l'ID : 31257
Bien enregistr